In [ ]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
pip install nlp-id

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 22.1 MB/s eta 0:00:00
  Created wheel for nlp-id: filename=nlp_id-0.1.15.0-py3-none-any.whl size=58153891 sha256=dd9284ee146ddcb98aa2aa7fdb4cd236b77377b1f64ab0e65c3ba16456177db1
  Stored in directory: /root/.cache/pip/wheels/85/34/2a/deaf7b7896a1eed336b874e1a7732588d40bb7bdc0a824e73e
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=deff317c084a518837d2a8d527f7f494abc572d1261d46ebb7eb220592194aa2
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built nlp-id wget
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.3
    Uninstalling pytest-7.4.3:
      Successfully uninstalled pytest-7.4.3


In [ ]:
import json
import random
import Sastrawi
import nltk
import warnings
import numpy as np
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from keras.models import load_model
from nlp_id.lemmatizer import Lemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = json.loads(open("/content/drive/MyDrive/indataset.json").read())

In [ ]:
data

{'intents': [{'tag': 'Luka (akibat benda tajam)',
   'patterns': ['Apa yang harus dilakukan jika terjadi luka akibat benda tajam?',
    'Bagaimana cara mengobati luka akibat benda tajam?',
    'Obat apa yang harus digunakan pada luka akibat benda tajam?',
    'Apa yang harus diberikan pada luka akibat benda tajam?',
    'Luka karena benda tajam'],
   'responses': ['Cuci luka dengan benar untuk mencegah infeksi, hentikan pendarahan dengan menekan lembut pada luka selama 1-2 menit hingga pendarahan berhenti, beri betadine, dan tutup dengan perban steril. Apabila luka terjadi pada alat gerak maka tinggikan anggota gerak yang luka hingga melebihi tinggi jantung untuk mengurangi perdarahan'],
   'context_set': ''},
  {'tag': 'Lecet',
   'patterns': ['Bagaimana penanganan pada lecet?',
    'Apakah lecet menyebabkan bekas?',
    'Lecet',
    'Apa yang dilakukan jika terjadi lecet?',
    'Obat apa yang digunakan untuk mengobati lecet?',
    'Bagaimana cara mengobati lecet?'],
   'responses': [

In [ ]:
# create stemmer
# factory = StemmerFactory()
# stemmer = factory.create_stemmer()
# lemmatizer = Lemmatizer()

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []
# ignore_words = [".", ",", "?", "!", ")", "(", " "]

In [ ]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
# Word Steamming
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [ ]:
# Bag of Words
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

In [ ]:
# Split Data
train_data, val_data, train_output, val_output = train_test_split(training, output, test_size=0.25, random_state=42)

In [ ]:
# Developing a Model
model = Sequential([
    Dense(32, input_shape=(len(training[0]),), activation='relu'),
    Dropout(0.4),
    Dense(16, activation='relu'),
    Dense(len(output[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history=model.fit(train_data, train_output, epochs=500, batch_size=32, validation_data=(val_data, val_output))
model.save('InChatBot_Model.h5', history)
accuracy = history.history['accuracy'][-1] * 100
print('\n')
print('*'*100)
print("\nPembuatan model sukses")
print(f"Akurasi sebesar: {accuracy:.2f}%")

Epoch 1/500
5/5 [==============================] - 3s 238ms/step - loss: 3.7625 - accuracy: 0.0145 - val_loss: 3.7400 - val_accuracy: 0.0217
Epoch 2/500
5/5 [==============================] - 0s 19ms/step - loss: 3.7526 - accuracy: 0.0290 - val_loss: 3.7396 - val_accuracy: 0.0435
Epoch 3/500
5/5 [==============================] - 0s 20ms/step - loss: 3.7287 - accuracy: 0.0217 - val_loss: 3.7395 - val_accuracy: 0.0435
Epoch 4/500
5/5 [==============================] - 0s 24ms/step - loss: 3.7311 - accuracy: 0.0290 - val_loss: 3.7385 - val_accuracy: 0.0435
Epoch 5/500
5/5 [==============================] - 0s 19ms/step - loss: 3.7263 - accuracy: 0.0435 - val_loss: 3.7379 - val_accuracy: 0.0435
Epoch 6/500
5/5 [==============================] - 0s 25ms/step - loss: 3.7210 - accuracy: 0.0580 - val_loss: 3.7365 - val_accuracy: 0.0435
Epoch 7/500
5/5 [==============================] - 0s 34ms/step - loss: 3.7093 - accuracy: 0.0290 - val_loss: 3.7348 - val_accuracy: 0.0435
Epoch 8/500
5/5 [==

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
